# Individual glacier analysis 2

This notebook will demonstrate the workflow for reading in glacier outlines (vector data) and glacier velocity (raster data) for a large region, extracting the data to examine an individual glacier and performing some preliminary analysis of glacier surface velocities at the scale of an individual glacier

We start by installing the necessary software:

In [ ]:
import xarray as xr
from skimage.morphology import skeletonize
from shapely.geometry import MultiPolygon
import geopandas as gpd
import rioxarray as rxr
import numpy as np
from matplotlib import pyplot as plt

Set paths (hopefully move away from this once accessing data from cloud)

In [ ]:
gen_path = '/Users/emarshall/Desktop/siparcs/data/'
rgi_path = '/Users/emarshall/Desktop/siparcs/data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'

In [ ]:
gen_path_home = '/Users/emmamarshall/Desktop/phd_research/siparcs/'
rgi_path_home = '/Users/emmamarshall/Downloads/15rgi60SouthAsiaEast'

## Read in raster data

In this example, we'll again by using ITS_LIVE ice velocity data downloaded from the NSIDC DAAC access point (for now)

In [ ]:
itslive = rxr.open_rasterio(gen_path + 'HMA_G0120_0000.nc').squeeze()

What CRS is this dataset in?

In [ ]:
itslive.rio.crs

In [ ]:
itslive.v.plot()

In [ ]:
itslive.v.UTM_Projection.GeoTransform

It looks like the ITS_LIVE data is in the Asia North Lambert Conformal Conic coordinate reference system. When we read in the vector data and begin to work with it we will need to project it to match this.

## Read in vector data

The glacier outline and attribute data we will be using is from the Randolph Glacier Inventory (RGI). The RGI contains data for glaciers worldwide and is divided into 19 primary regions. We will read in one region (South Asia East) here. 

In [ ]:
#se_asia = gpd.read_file(gen_path_home[:-9] + 'data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')
se_asia = gpd.read_file(rgi_path)


And let's explore this data a bit by looking at:
- the number of glaciers within the dataset
- the spatial extent of the dataset
- the coordinate reference system of the dataset


In [ ]:
print('Number of glaciers in dataset: ', len(se_asia['RGIId']))

In [ ]:
se_asia.plot()

In [ ]:
print('CRS: ', se_asia.crs)

The RGI glacier outline dataframe is in lat/lon so let's project it to match the raster data:
If you look at the geoemtry column of the projected dataset you will see that the coordinates are now in meters instead of degrees.

In [ ]:
se_asia_prj = se_asia.to_crs('+proj=lcc +lat_1=15 +lat_2=65 +lat_0=30 +lon_0=95 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs')
se_asia_prj

## Clipping raster data by a vector

Now, let's focus on just a single glacier. To start with, let's look at the five largest (in area) glaciers in the dataset and choose one of those.

In [ ]:
se_asia.sort_values(by = 'Area', ascending=False).head()

To start with, let's look at the glacier with the RGIId RGI60-15.11926 and an area of 96.283 km2. 

In [ ]:
rgi11926 = se_asia_prj.loc[se_asia_prj['RGIId'] == 'RGI60-15.11926']
rgi11926

In [ ]:
raster_11926 = itslive.rio.clip(rgi11926.geometry, rgi11926.crs)

In [ ]:
#turn non-ice pixels to NaN (previously -32767.)
clipped_raster11926 = xr.where(raster_11926.ice == 1., raster_11926, np.NaN)

## Extract glacier centerline

- using scikit-image skeletonize()
- probably not the best/ideal way to do this, for now wanted to have an example but maybe not worth including here? 

In [ ]:
def extract_glacier_centerline(glacier_gpdf, full_scene_raster):
    ''' This function should take in a vector outline of a single glacier and a xr object of glacier velocity data (full scene) 
    and return a dictionary (?) with 2 keys and values corresponding to two xarray objects: one showing the full 2d extent of 
    raster data clipped to the glacier outline and one with the extracted glacier centerline'''

    #clip raster by vector
    clipped_raster = full_scene_raster.rio.clip(glacier_gpdf.geometry, glacier_gpdf.crs)

    #turn non-ice pixels to NaN (previously -32767.)
    clipped_raster = xr.where(clipped_raster.ice == 1., clipped_raster, np.NaN)

    #extract centerline - this is a xr DataArray that only has an boolean array representing centerline
    centerline_da = clipped_raster.ice.reduce(skeletonize)

    #create xarray dataset with centerline extracted for all data vars
    centerline_ds = clipped_raster.where(centerline_da != False)

    #wrap into dict
    glacier_dict = {'clipped_to_ice': clipped_raster,
                    'centerline': centerline_ds
                    }

    return glacier_dict

In [ ]:
g11926_dict = extract_glacier_centerline(rgi11926, itslive)
g11926_cl = g11926_dict['centerline']
g11926_cl.v.plot()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize= (14,7))

g11926_cl.v.plot(ax=axs[0])
g11926_dict['clipped_to_ice'].v.plot(ax=axs[1])

In [ ]:
g06681_dict = extract_glacier_centerline(rgi06681, itslive)

### non-function examples below

In [ ]:
centerline_11926_da = clipped_raster11926.ice.reduce(skeletonize)
centerline_11926_da.plot()
type(centerline_11926_da)

In [ ]:
centerline_11926_ds = clipped_raster11926.where(centerline_11926_da != False)
centerline_11926_ds.v.plot()

In [ ]:
g11926_dict = {'full':raster_11926,
                'clipped_to_ice': clipped_raster11926,
                'centerline': centerline_11926_ds}

In [ ]:
g11926_dict = extract_glacier_centerline(rgi11926, itslive)
g11926_cl = g11926_dict['centerline']
g11926_cl.v.plot()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize= (14,7))

g11926_cl.v.plot(ax=axs[0])
g11926_dict['clipped_to_ice'].v.plot(ax=axs[1])

In [ ]:
g06681_dict = extract_glacier_centerline(rgi06681, itslive)

In [ ]:
g06681_dict.keys()

In [ ]:
g06681_cl = g06681_dict['centerline']
g06681_cl.v.plot()

In [ ]:
rgi06681 = se_asia_prj.loc[se_asia_prj['RGIId'] == 'RGI60-15.06681']
se_asia_prj.sort_values('Area', ascending=False).head()

### A second glacier

In [ ]:
rgi11909 = se_asia_prj.loc[se_asia_prj['RGIId'] == 'RGI60-15.11909']
rgi11909

In [ ]:
raster_11909 = itslive.rio.clip(rgi11909.geometry, rgi11909.crs)

In [ ]:
#turn non-ice pixels to NaN ()
clipped_raster11909 = xr.where(raster_11909.ice == 1., raster_11909, np.NaN)

Compare different skeletonize algorithms:

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (14,4), sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(just_ice.v, cmap=plt.cm.cividis)
ax[0].axis('off')
ax[0].set_title('Glacier velocity', fontsize=14)
ax[0].imshow(skeleton_ice, cmap=plt.cm.Reds, alpha = 0.2)
ax[0].imshow(skeleton_ice_lee, cmap=plt.cm.Greens, alpha = 0.2)


ax[1].imshow(skeleton_ice, cmap=plt.cm.Reds)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=14)

ax[2].imshow(skeleton_ice_lee, cmap=plt.cm.Greens)
ax[2].axis('off')
ax[2].set_title('skeleton (lee)', fontsize=14)

fig.tight_layout()
fig.show

## Make xarray object out of centerline bool mask
- should have the same coords as the velocity xarray object

In [ ]:
centerline_da = xr.DataArray(
    skeleton_ice,
    dims=("y", "x"),
    coords={
        "x": glacier_raster.coords['x'].data,
        "y": glacier_raster.coords['y'].data}
)


centerline_da.plot()

In [ ]:
centerline_da

### Extract itslive xarray dataset data only at points along centerline
- this should return a dataset with the same data vars but only data along centerline

In [ ]:
centerline_ds = glacier_raster.where(da != False)

In [ ]:
centerline_ds

In [ ]:
fig, axs = plt.subplots(ncols = 2, figsize=(20,8))
just_ice.ice.plot(ax=axs[0], cmap=plt.cm.cividis)
centerline_ds.v.plot(ax=axs[0])

just_ice.v.plot(ax=axs[1])

### Experimenting with skeletonize v. thin (scikit image)

In [ ]:
just_ice.ice.plot()

In [ ]:
from skimage.morphology import skeletonize, thin

thinned = thin(just_ice.ice)
#thinned_partial = thin(just_ice.ice.data, max_num_iter = 50)

plt.imshow(thinned)

In [ ]:
fig, axs = plt.subplots(ncols=2)

axs[0].imshow(thinned, cmap=plt.cm.gray)
axs[0].set_title('thinned')
axs[0].axis('off')

axs[1].imshow(thinned_partial, cmap=plt.cm.gray)
axs[1].set_title('thinned partial')
axs[1].axis('off')

